# ***Poly*, a go package for organism engineering** 

***Poly*** is a fast and modern project created by [**@TimothyStiles**](https://twitter.com/TimothyStiles) to resolve different problems found while engineering living things. From designing primers for barcoding to CDS fixing, Poly is trying to be the most complete, open and well-used computational synthetic biology tool.


---


- Poly is our key tool for these pipelines. It was a planned decision to build workflows that integrate with this package, to show ways to use the package, as well as create some new features; therefore, it is very important that you understand how the Poly package works and what its structure is in general before you begin manipulating it. Thus, we created this brief overview of Poly, its sub-packages, and a collection of use cases. 
- We strongly recommend that you do the tutorials in the order they appear.
- These notebooks were made to run on Google Colaboratory.

---

Authors: [**@GiovannaMaklouf**](https://twitter.com/giomaklouf), [**@IsaacG**](https://twitter.com/IsaacGuerreiro9)


# Configurations for this tutorial




First let's run some important settings so you can run this tutorial successfully. 


Colab notebooks use python kernels to run each cell. However, because ***Poly*** is written in **Go language (golang)**, we need to install and configure some things in colab to make feasible run something in go lang here.

### **1. In order to start the golang environment, run the line below:**

In [ ]:
# this process may take a few minutes
!add-apt-repository ppa:longsleep/golang-backports -y
!apt update
!apt install golang-go
%env GOPATH=/root/go
!go get -u github.com/gopherdata/gophernotes
!cp ~/go/bin/gophernotes /usr/bin/
!npx degit gopherdata/gophernotes/kernel \
     /usr/local/share/jupyter/kernels/gophernotes

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [69.5 kB]
Hit:12 http://ppa.launchpad.net/cran/

### **2. Download important data to run this tutorial**

In [ ]:
!rm -rf $GOPATH/pkg/mod/github.com/!open-!science-!global
!rm -rf $GOPATH/pkg/mod/cache/download/github.com/!open-!science-!global
!go get -u github.com/Open-Science-Global/poly@e3e1c61

go: downloading github.com/Open-Science-Global/poly v0.11.3


In [ ]:
!wget https://raw.githubusercontent.com/Open-Science-Global/friendzymes-cookbook/main/data/understanding-poly/base.fasta
!wget https://raw.githubusercontent.com/Open-Science-Global/friendzymes-cookbook/main/data/understanding-poly/puc19.gbk

### **3. Connect Colab Notebook to your GDrive (not required)**

The previous code will download the files temporarily. If you want to download them to a folder on your drive and save it for later analysis or if you are already using this notebook to run your own files, you should connect your Google Drive to Colab and you will be able to access, read and save files permanently. 
So, if you prefer, you can do this with the code line below:



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## After running these steps, click on **Runtime** in the menu bar & **Change Runtime Type** to Go, if it hasn't changed yet.

This will make Colab starting use a open source go kernel called Gopher Notes.

Now we are ready to work.

# **Importing Poly**

The first thing to know is that we created a fork of Poly for our Friendzymes repository to implements new features and improvements fast. You could take a look in the project [**here**](https://github.com/Open-Science-Global/poly).


## **A tour of Poly's key subpackages**

Poly has features divided by subpackages. Each subpackage has functions and data structures related to some topic, e.g. clone, synthesis, primers, random, transform, io and others.

To import a subpackage you could use like the example below for the **subpackage random** and golang will know what you want.

In [ ]:
import "github.com/Open-Science-Global/poly/random"

// example of use of the random subpackage
randomSequence, err := random.ProteinSequence(100, 1)
randomSequence

MCIIYCWGATARNDLJRNGVHSHJWIIIJMSCJINLTVNGHQMREQIWFYQVCLYAGJWESNMGTHJCDEHETDWIRVETAEQVQCYQENDWTHIAEPE*

## IO (Input Output)

IO is the subpackage related to reading and writing of files extensions commonly used in synthetic biology. Poly could deal with the following file extensions:



*   **Genbank**
*   **FASTA**
*   **Poly Json** (a new proposal to parse genetic sequences)
*   **GFF**
*   **Rebase**
*   **Uniprot**



In [ ]:
import (
  "fmt"
  "github.com/Open-Science-Global/poly/io/fasta"
  "github.com/Open-Science-Global/poly/io/genbank"
)

fmt.Println("#Read a fasta file and show the sequences:")

bases := fasta.Read("./base.fasta")
for i, v := range(bases) {
  fmt.Println(v)
}

fmt.Println()
fmt.Println("#Read a genbank file and show the sequences:")

puc19 := genbank.Read("./puc19.gbk")
fmt.Println("GenBank Sequence", "|", puc19.Sequence)

for i, feature := range(puc19.Features) {
  if (feature.Type == "CDS") {
    fmt.Println("CDS", "|", feature.Attributes["product"], "|", feature.Attributes["translation"])
  }
}

#Read a fasta file and showing the sequences:
{ }

#Read a genbank file and showing the sequences:
GenBank Sequence | 


## Transform


The transform subpackage deal with some *transformation* as get a **complement** or **reverse complement** of some DNA sequence. It also have another subpackage called ***codons*** to deal with **translation** and **codon optimization**.

In [ ]:
import (
  "fmt"
  "github.com/Open-Science-Global/poly/transform"
  "github.com/Open-Science-Global/poly/transform/codon"
)

seq := "GATTACA"
fmt.Println("Sequence:", seq, "Complement:", transform.Complement(seq), "Reverse Complement:", transform.ReverseComplement(seq))

gfp := "ATGGCTAGCAAAGGAGAAGAACTTTTCACTGGAGTTGTCCCAATTCTTGTTGAATTAGATGGTGATGTTAATGGGCACAAATTTTCTGTCAGTGGAGAGGGTGAAGGTGATGCTACATACGGAAAGCTTACCCTTAAATTTATTTGCACTACTGGAAAACTACCTGTTCCATGGCCAACACTTGTCACTACTTTCTCTTATGGTGTTCAATGCTTTTCCCGTTATCCGGATCATATGAAACGGCATGACTTTTTCAAGAGTGCCATGCCCGAAGGTTATGTACAGGAACGCACTATATCTTTCAAAGATGACGGGAACTACAAGACGCGTGCTGAAGTCAAGTTTGAAGGTGATACCCTTGTTAATCGTATCGAGTTAAAAGGTATTGATTTTAAAGAAGATGGAAACATTCTCGGACACAAACTCGAGTACAACTATAACTCACACAATGTATACATCACGGCAGACAAACAAAAGAATGGAATCAAAGCTAACTTCAAAATTCGCCACAACATTGAAGATGGATCCGTTCAACTAGCAGACCATTATCAACAAAATACTCCAATTGGCGATGGCCCTGTCCTTTTACCAGACAACCATTACCTGTCGACACAATCTGCCCTTTCGAAAGATCCCAACGAAAAGCGTGACCACATGGTCCTTCTTGAGTTTGTAACTGCTGCTGGGATTACACATGGCATGGATGAGCTCTACAAATAA"
fmt.Println("You could translate a sequence using codon.Translate passing the sequence AND a referenced codon table")
translation, _ := codon.Translate(gfp, codon.GetCodonTable(11))
fmt.Println("GFP Translated:", translation)

fmt.Println()
fmt.Println("You could optimize a sequence using codon.Optimize passing the sequence AND optimized codon table")
optimized, _ := codon.Optimize(gfp, codon.GetCodonTable(11).OptimizeTable(gfp))
fmt.Println("GFP Codon Optimization:", optimized)

Sequence: GATTACA Complement: CTAATGT Reverse Complement: TGTAATC
You could translate a sequence using codon.Translate passing the sequence AND a referenced codon table
GFP Translated: MASKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTFSYGVQCFSRYPDHMKRHDFFKSAMPEGYVQERTISFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYITADKQKNGIKANFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK*

You could optimize a sequence using codon.Optimize passing the sequence AND optimized codon table
GFP Codon Optimization: GCTACAGGAGGCTGCACTGCAGGGTGCGCTGCCGCAGGAGGTGCTGGAGCTGCTGGAGCAGCCTGCACTACTACAACATGCGCTTGCACAGGAGGAGCTGGCACTACTGGAACATGCTGCTGCGCTGCAACAACCTGCACTACCGGAACAACGGGAGCAGCTACTACTGCCGGTGCTACTGGAGGTACCGGAGCTACTGGAACAACTGCTGCTACAGGAGGAGGGTGCGCATGCGCTGCTGCAACGACTACTACTTGCACCGGAACATGCGCAGGTACTGGAGGGGCAGGTGCCGGTGGCGGTACAGGAGCTGCAGGAGGAACTGGCGCTACGGGTTGCACTGCTTGCGCTACCGCTTGCGGCGGTGCTGCTGCTGGTTGCACTACAGCTTGCTGCTGCACGACTGCTGCTGCTACAACGACAGCCACTACTACTGGCTGCGCTTGCACAG

2185 <nil>

## Clone

The Clone subpackage has two main functionalies: 
- 1) a function that simulates restriction digestion and returns a list of fragments (linear and circular sequences)
  
- **Important:** every time we have "false" as parameter within the variable it means that this is not from a circular sequence. For example: `cloned := clone.Part{mySeq1 + bsaiBindingSite + mySeq2, false} `. When we have "true" it means that it comes from a circular sequence like a plasmid.

In [ ]:
import (
  "fmt"
  "github.com/Open-Science-Global/poly/clone"
)
bsaiBindingSite := "GGTCTCAATGC"
mySeq1 := "ATATATATATATATAT"
mySeq2 := "GCGCGCGCGCGCGCGCGCGC"
cloned := clone.Part{mySeq1 + bsaiBindingSite + mySeq2, false} // false means it's not from a circular sequence
cloned

{ATATATATATATATATGGTCTCAATGCGCGCGCGCGCGCGCGCGCGC false}

In [ ]:
fragments, err := clone.CutWithEnzymeByName(cloned, false, "BsaI")

fragments

[{GCGCGCGCGCGCGCGCGCGC ATGC } {ATATATATATATATATGGTCTCA  ATGC}]

- 2) a Golden Gate Assembly simulator

The variable `popen` is the sequence of the plasmid (note that it is with the "true" parameter), `fragment1` and `fragment2` are the parts I want to clone. Then this is passed to the `GoldenGate()` function with the indication of the enzyme we want to simulate, which in this case is BbsI.


In [ ]:
var popen = clone.Part{"TAACTATCGTCTTGAGTCCAACCCGGTAAGACACGACTTATCGCCACTGGCAGCAGCCACTGGTAACAGGATTAGCAGAGCGAGGTATGTAGGCGGTGCTACAGAGTTCTTGAAGTGGTGGCCTAACTACGGCTACACTAGAAGAACAGTATTTGGTATCTGCGCTCTGCTGAAGCCAGTTACCTTCGGAAAAAGAGTTGGTAGCTCTTGATCCGGCAAACAAACCACCGCTGGTAGCGGTGGTTTTTTTGTTTGCAAGCAGCAGATTACGCGCAGAAAAAAAGGATCTCAAGAAGGCCTACTATTAGCAACAACGATCCTTTGATCTTTTCTACGGGGTCTGACGCTCAGTGGAACGAAAACTCACGTTAAGGGATTTTGGTCATGAGATTATCAAAAAGGATCTTCACCTAGATCCTTTTAAATTAAAAATGAAGTTTTAAATCAATCTAAAGTATATATGAGTAAACTTGGTCTGACAGTTACCAATGCTTAATCAGTGAGGCACCTATCTCAGCGATCTGTCTATTTCGTTCATCCATAGTTGCCTGACTCCCCGTCGTGTAGATAACTACGATACGGGAGGGCTTACCATCTGGCCCCAGTGCTGCAATGATACCGCGAGAACCACGCTCACCGGCTCCAGATTTATCAGCAATAAACCAGCCAGCCGGAAGGGCCGAGCGCAGAAGTGGTCCTGCAACTTTATCCGCCTCCATCCAGTCTATTAATTGTTGCCGGGAAGCTAGAGTAAGTAGTTCGCCAGTTAATAGTTTGCGCAACGTTGTTGCCATTGCTACAGGCATCGTGGTGTCACGCTCGTCGTTTGGTATGGCTTCATTCAGCTCCGGTTCCCAACGATCAAGGCGAGTTACATGATCCCCCATGTTGTGCAAAAAAGCGGTTAGCTCCTTCGGTCCTCCGATCGTTGTCAGAAGTAAGTTGGCCGCAGTGTTATCACTCATGGTTATGGCAGCACTGCATAATTCTCTTACTGTCATGCCATCCGTAAGATGCTTTTCTGTGACTGGTGAGTACTCAACCAAGTCATTCTGAGAATAGTGTATGCGGCGACCGAGTTGCTCTTGCCCGGCGTCAATACGGGATAATACCGCGCCACATAGCAGAACTTTAAAAGTGCTCATCATTGGAAAACGTTCTTCGGGGCGAAAACTCTCAAGGATCTTACCGCTGTTGAGATCCAGTTCGATGTAACCCACTCGTGCACCCAACTGATCTTCAGCATCTTTTACTTTCACCAGCGTTTCTGGGTGAGCAAAAACAGGAAGGCAAAATGCCGCAAAAAAGGGAATAAGGGCGACACGGAAATGTTGAATACTCATACTCTTCCTTTTTCAATATTATTGAAGCATTTATCAGGGTTATTGTCTCATGAGCGGATACATATTTGAATGTATTTAGAAAAATAAACAAATAGGGGTTCCGCGCACCTGCACCAGTCAGTAAAACGACGGCCAGTAGTCAAAAGCCTCCGACCGGAGGCTTTTGACTTGGTTCAGGTGGAGTGGGAGTAgtcttcGCcatcgCtACTAAAagccagataacagtatgcgtatttgcgcgctgatttttgcggtataagaatatatactgatatgtatacccgaagtatgtcaaaaagaggtatgctatgaagcagcgtattacagtgacagttgacagcgacagctatcagttgctcaaggcatatatgatgtcaatatctccggtctggtaagcacaaccatgcagaatgaagcccgtcgtctgcgtgccgaacgctggaaagcggaaaatcaggaagggatggctgaggtcgcccggtttattgaaatgaacggctcttttgctgacgagaacagggGCTGGTGAAATGCAGTTTAAGGTTTACACCTATAAAAGAGAGAGCCGTTATCGTCTGTTTGTGGATGTACAGAGTGATATTATTGACACGCCCGGGCGACGGATGGTGATCCCCCTGGCCAGTGCACGTCTGCTGTCAGATAAAGTCTCCCGTGAACTTTACCCGGTGGTGCATATCGGGGATGAAAGCTGGCGCATGATGACCACCGATATGGCCAGTGTGCCGGTCTCCGTTATCGGGGAAGAAGTGGCTGATCTCAGCCACCGCGAAAATGACATCAAAAACGCCATTAACCTGATGTTCTGGGGAATATAAATGTCAGGCTCCCTTATACACAGgcgatgttgaagaccaCGCTGAGGTGTCAATCGTCGGAGCCGCTGAGCAATAACTAGCATAACCCCTTGGGGCCTCTAAACGGGTCTTGAGGGGTTTTTTGCATGGTCATAGCTGTTTCCTGAGAGCTTGGCAGGTGATGACACACATTAACAAATTTCGTGAGGAGTCTCCAGAAGAATGCCATTAATTTCCATAGGCTCCGCCCCCCTGACGAGCATCACAAAAATCGACGCTCAAGTCAGAGGTGGCGAAACCCGACAGGACTATAAAGATACCAGGCGTTTCCCCCTGGAAGCTCCCTCGTGCGCTCTCCTGTTCCGACCCTGCCGCTTACCGGATACCTGTCCGCCTTTCTCCCTTCGGGAAGCGTGGCGCTTTCTCATAGCTCACGCTGTAGGTATCTCAGTTCGGTGTAGGTCGTTCGCTCCAAGCTGGGCTGTGTGCACGAACCCCCCGTTCAGCCCGACCGCTGCGCCTTATCCGG", true} // true means it's circular
fragment1 := clone.Part{"GAAGTGCCATTCCGCCTGACCTGAAGACCAGGAGAAACACGTGGCAAACATTCCGGTCTCAAATGGAAAAGAGCAACGAAACCAACGGCTACCTTGACAGCGCTCAAGCCGGCCCTGCAGCTGGCCCGGGCGCTCCGGGTACCGCCGCGGGTCGTGCACGTCGTTGCGCGGGCTTCCTGCGGCGCCAAGCGCTGGTGCTGCTCACGGTGTCTGGTGTTCTGGCAGGCGCCGGTTTGGGCGCGGCACTGCGTGGGCTCAGCCTGAGCCGCACCCAGGTCACCTACCTGGCCTTCCCCGGCGAGATGCTGCTCCGCATGCTGCGCATGATCATCCTGCCGCTGGTGGTCTGCAGCCTGGTGTCGGGCGCCGCCTCCCTCGATGCCAGCTGCCTCGGGCGTCTGGGCGGTATCGCTGTCGCCTACTTTGGCCTCACCACACTGAGTGCCTCGGCGCTCGCCGTGGCCTTGGCGTTCATCATCAAGCCAGGATCCGGTGCGCAGACCCTTCAGTCCAGCGACCTGGGGCTGGAGGACTCGGGGCCTCCTCCTGTCCCCAAAGAAACGGTGGACTCTTTCCTCGACCTGGCCAGAAACCTGTTTCCCTCCAATCTTGTGGTTGCAGCTTTCCGTACGTATGCAACCGATTATAAAGTCGTGACCCAGAACAGCAGCTCTGGAAATGTAACCCATGAAAAGATCCCCATAGGCACTGAGATAGAAGGGATGAACATTTTAGGATTGGTCCTGTTTGCTCTGGTGTTAGGAGTGGCCTTAAAGAAACTAGGCTCCGAAGGAGAGGACCTCATCCGTTTCTTCAATTCCCTCAACGAGGCGACGATGGTGCTGGTGTCCTGGATTATGTGGTACGCGTCTTCAGGCTAGGTGGAGGCTCAGTG", false}
fragment2 := clone.Part{"GAAGTGCCATTCCGCCTGACCTGAAGACCAGTACGTACCTGTGGGCATCATGTTCCTTGTTGGAAGCAAGATCGTGGAAATGAAAGACATCATCGTGCTGGTGACCAGCCTGGGGAAATACATCTTCGCATCTATATTGGGCCACGTCATTCATGGTGGTATCGTCCTGCCGCTGATTTATTTTGTTTTCACACGAAAAAACCCATTCAGATTCCTCCTGGGCCTCCTCGCCCCATTTGCGACAGCATTTGCTACGTGCTCCAGCTCAGCGACCCTTCCCTCTATGATGAAGTGCATTGAAGAGAACAATGGTGTGGACAAGAGGATCTCCAGGTTTATTCTCCCCATCGGGGCCACCGTGAACATGGACGGAGCAGCCATCTTCCAGTGTGTGGCCGCGGTGTTCATTGCGCAACTCAACAACGTAGAGCTCAACGCAGGACAGATTTTCACCATTCTAGTGACTGCCACAGCGTCCAGTGTTGGAGCAGCAGGCGTGCCAGCTGGAGGGGTCCTCACCATTGCCATTATCCTGGAGGCCATTGGGCTGCCTACTCATGATCTGCCTCTGATCCTGGCTGTGGACTGGATTGTGGACCGGACCACCACGGTGGTGAATGTGGAAGGGGATGCCCTGGGTGCAGGCATTCTCCACCACCTGAATCAGAAGGCAACAAAGAAAGGCGAGCAGGAACTTGCTGAGGTGAAAGTGGAAGCCATCCCCAACTGCAAGTCTGAGGAGGAAACCTCGCCCCTGGTGACACACCAGAACCCCGCTGGCCCCGTGGCCAGTGCCCCAGAACTGGAATCCAAGGAGTCGGTTCTGTGAAGAGCTTAGAGACCGACGACTGCCTAAGGACATTCGCTGCGTCTTCAGGCTAGGTGGAGGCTCAGTG", false}

Clones, _ := clone.GoldenGate([]clone.Part{fragment1, fragment2, popen}, "BbsI")
Clones

[{GGAGAAACACGTGGCAAACATTCCGGTCTCAAATGGAAAAGAGCAACGAAACCAACGGCTACCTTGACAGCGCTCAAGCCGGCCCTGCAGCTGGCCCGGGCGCTCCGGGTACCGCCGCGGGTCGTGCACGTCGTTGCGCGGGCTTCCTGCGGCGCCAAGCGCTGGTGCTGCTCACGGTGTCTGGTGTTCTGGCAGGCGCCGGTTTGGGCGCGGCACTGCGTGGGCTCAGCCTGAGCCGCACCCAGGTCACCTACCTGGCCTTCCCCGGCGAGATGCTGCTCCGCATGCTGCGCATGATCATCCTGCCGCTGGTGGTCTGCAGCCTGGTGTCGGGCGCCGCCTCCCTCGATGCCAGCTGCCTCGGGCGTCTGGGCGGTATCGCTGTCGCCTACTTTGGCCTCACCACACTGAGTGCCTCGGCGCTCGCCGTGGCCTTGGCGTTCATCATCAAGCCAGGATCCGGTGCGCAGACCCTTCAGTCCAGCGACCTGGGGCTGGAGGACTCGGGGCCTCCTCCTGTCCCCAAAGAAACGGTGGACTCTTTCCTCGACCTGGCCAGAAACCTGTTTCCCTCCAATCTTGTGGTTGCAGCTTTCCGTACGTATGCAACCGATTATAAAGTCGTGACCCAGAACAGCAGCTCTGGAAATGTAACCCATGAAAAGATCCCCATAGGCACTGAGATAGAAGGGATGAACATTTTAGGATTGGTCCTGTTTGCTCTGGTGTTAGGAGTGGCCTTAAAGAAACTAGGCTCCGAAGGAGAGGACCTCATCCGTTTCTTCAATTCCCTCAACGAGGCGACGATGGTGCTGGTGTCCTGGATTATGTGGTACGTACCTGTGGGCATCATGTTCCTTGTTGGAAGCAAGATCGTGGAAATGAAAGACATCATCGTGCTGGTGACCAGCCTGGGGAAATACATCTTCGCATCTATATTGGGCCACGTCATTCATGGTGGTATCGTCCTGCCGCTGATTTATTTTGTTTTCACACGA

## Synthesis

The Synthesis subpackage has one main feature called `fixCDS`. Coding sequences give us the opportunity to change codons to 
optimize the sequence in a lot of ways like removing undesirables sequences, optimizing codon usage, and lowering GC content. `fixCDS` give us the opportunity to make suggestions of changes and optimize this genetic part.

In [ ]:
import (
  "github.com/Open-Science-Global/poly/synthesis"
  "sync"
)

seq := "ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGGCATTTTGCCTTCCTGTTTTTGCTCACCCAGAAACGCTGGTGAAAGTAAAAGATGCTGAAGATCAGTTGGGTGCACGAGTGGGTTACATCGAACTGGATCTCAACAGCGGTAAGATCCTTGAGAGTTTTCGCCCCGAAGAACGTTTTCCAATGATGAGCACTTTTAAAGTTCTGCTATGTGGCGCGGTATTATCCCGTATTGACGCCGGGCAAGAGCAACTCGGTCGCCGCATACACTATTCTCAGAATGACTTGGTTGAGTACTCACCAGTCACAGAAAAGCATCTTACGGATGGCATGACAGTAAGAGAATTATGCAGTGCTGCCATAACCATGAGTGATAACACTGCGGCCAACTTACTTCTGACAACGATCGGAGGACCGAAGGAGCTAACCGCTTTTTTGCACAACATGGGGGATCATGTAACTCGCCTTGATCGTTGGGAACCGGAGCTGAATGAAGCCATACCAAACGACGAGCGTGACACCACGATGCCTGTAGCAATGGCAACAACGTTGCGCAAACTATTAACTGGCGAACTACTTACTCTAGCTTCCCGGCAACAATTAATAGACTGGATGGAGGCGGATAAAGTTGCAGGACCACTTCTGCGCTCGGCCCTTCCGGCTGGCTGGTTTATTGCTGATAAATCTGGAGCCGGTGAGCGTGGGTCTCGCGGTATCATTGCAGCACTGGGGCCAGATGGTAAGCCCTCCCGTATCGTAGTTATCTACACGACGGGGAGTCAGGCAACTATGGATGAACGAAATAGACAGATCGCTGAGATAGGTGCCTCACTGATTAAGCATTGGTAA"
sequence := genbank.Read("data/ecoli-mg1655.gff")
codonTable := codon.GetCodonTable(11)
codingRegions := codon.GetCodingRegions(sequence)
optimizationTable := codonTable.OptimizeTable(codingRegions)

var functions []func(string, chan synthesis.DnaSuggestion, *sync.WaitGroup)
functions = append(functions, synthesis.RemoveSequence([]string{"GAAGAC", "GGTCTC", "GCGATG", "CGTCTC", "GCTCTTC", "CACCTGC"}))

fixedSeq, _ := synthesis.FixCds(":memory:", seq, optimizationTable, functions)
fixedSeq

ERROR: ignored

## Primer

The Primers subpackage includes a 
- 1) Melting Temperature (Tm) Calculator

In [ ]:
import (
  "fmt"
  "github.com/Open-Science-Global/poly/primers"
)

primer := "GTAAAACGACGGCCAGT" //M13 fwd
primers.MeltingTemp(primer)

52.63382276100299

- 2) and a function for creating barcodes if your purpose is sequecing

In [ ]:
barcodes := primers.CreateBarcodes(20, 4)
barcodes

[AAAATAAAGAAACAATTAAT AATGAATCAAGTAAGGAAGC AGCAACTAACGAACCATATA ATAGATACATTTATTGATTC TTCATGTATGGATGCATCTA CTATCGATCCAGAGACAGTT GTTAGTGAGTCAGGTAGGGA GGAGGCAGCTAGCGAGCCAC CACACTTACTGACTCACGTA GTACGGACGCACCTACCGAC GACCCTTTTGTTTCTTGGTT GTTGCTTCGTTCCTGTGTCT TCTGGGTGGCTGCGTGCCTC CTCTCGGTCGCTCCGTCCCG CCGGGGCGGCCGCGCCCCAA]

## Other subpackages

Other subpackages include Random and Seqhash. We highly recommend taking a look to the highly readable and commented **[source code](https://github.com/Open-Science-Global/poly)**, and also the **[documentation](https://pkg.go.dev/github.com/TimothyStiles/poly)** with examples based on real tests.

Several of these subpackages will be looked at in more detail in the following tutorials.